Chapitre 5 - Créer un outil CLI
===

# CLI ?

<img src="images/cli.png" align="right" width="200px" alt="Image de terminal" />

CLI (ou __C__ ommand __L__ ine __I__ nterface) s'oppose à GUI (__G__ raphical __U__ ser __I__ nterface) : c'est un outil dont l'utilisation se fera dans un terminal. Ce genre d'interface, quoiqu'abrupte pour les débutant-e-s est bien souvent nécessaire quand l'on travaille sur des serveurs distants : ces serveurs possèdent rarement une interface graphique et seul le terminal est accessible.

Par ailleurs, le développement d'interface graphique peut être très lourd dans les langages natifs, à tel point que bien souvent, aujourd'hui, on a recours à des outils comme [Electron](https://electronjs.org/) qui est en fait une installation simpliste de Chromium et a donc recours à JavaScript et HTML pour créer leur design.

Alors, dans la vie de tous les jours, pourquoi avoir un script accessible en ligne de commande ? Et bien tout simplement pour accélérer quelques tâches ingrates du quotidien, ou pour éviter à avoir à écrire un fichier python avec des paramètres changeant. Car, qui dit interface, dit bien souvent paramétrisation : l'interface développée permettra de personnaliser certains paramètres (où enregistrer le résultat d'un téléchargement par exemple).



# Développer un CLI

## Avant-propos

Dans ce cours, nous allons voir l'utilisation de [`Click`](http://click.palletsprojects.com/), une librairie à installer (dans votre environnement, faites `pip install click`). Qui dit librairie à installer dit généralement compétition et choix. Il existe donc d'autres librairies et je vous renvoie à cet article [*Building Beautiful Command Line Interfaces with Python*](https://codeburst.io/building-beautiful-command-line-interfaces-with-python-26c7e1bb54df) de Oyetoke Tobi Emmanuel dont une archive PDF est disponible dans le repository.

Le plus important de ces concurrents est *argparser* qui a l'avantage d'être, comme `json` et `csv`, une librairie de base de python et qui ne nécessite donc aucune installation. Une traduction du tutoriel officiel de python est disponible : https://docs.python.org/fr/3.5/howto/argparse.html

Le choix de Click s'est fait pour deux raisons : c'est la librairie utilisée par Flask, la librairie de création de sites web que nous utiliserons plus tard; elle gère les groupes de commande et possède une syntaxe très agréable.

## De Jupyter à l'IDE

À partir de maintenant, nous utiliserons un IDE (**I**nterface de **D**éveloppement **I**ntégrée). En tant qu'étudiant-e-s, vous avez accès à [PyCharm](https://www.jetbrains.com/pycharm/?fromMenu) pour lequel il existe une version *Community* (gratuite, limitée, suffisante) et une version Pro (accès via un compte éducation sinon paiement de licence).

Nous lancerons nos outils depuis le terminal et nous construirons peu à peu un outil permettant de faire une recherche rapide dans un fichier CSV représentant un texte lemmatisé.

## Commençons !

Pour créer un script, nous allons déjà créer le fichier : nous l'appellerons "recherche.py". Ce script nous permettra de faire des recherches sur Isidore. Dans ce but, nous lirons un peu plus tard la documentation de l'API d'[Isidore](https://isidore.science/api).

Un bon script python prend la forme suivante : 

1. On importe les librairies nécessaires;
2. On écrit ses fonctions et ses variables globales (utilisées dans plusieurs fonctions potentiellement);
3. On écrit les fonctions d'appel (généralement, si on en a qu'une, on l'appellera simplement `run` ou `cli`)
4. On appelle cette fonction dans une condition qui vérifie si le fichier python est exécuté comme fichier principal (et non importé d'ailleurs)

```python
import click
import requests
import math
import csv

ISIDORE = "https://api.isidore.science/resource/search"


def parser_reponse_isidore(data):
    """ Fait une recherche sur Isidore

    :param data: JSON Parsed Data
    :type q: dict
    :returns: Tuple (
        Nombre de Résultats,
        Nombre de Pages,
        Liste de résultat sous forme de dictionnaire {title, desc, author, date}
    )
    """
    # On récupère le nombre de résultats
    nb_items = int(data["response"]["replies"]["meta"]["@items"])
    # On récupère le nombre de résultats par page
    items_per_page = int(data["response"]["replies"]["meta"]["@pageItems"])
    # Le nombre total de pages est l'arrondi supérieur de la division nb_items / items_per_page
    total_page = math.ceil(nb_items / items_per_page)

    # On crée une liste vide dans laquelle on enregistrera les données
    items = []
    # Pour chaque réponse
    for item in data["response"]["replies"]["content"]["reply"]:
        # On ajoute à items un nouvel objet
        items.append({
            "uri": item["@uri"],
            "title": item["isidore"]["title"],
            # dictionnaire.get(cle, valeur-par-defaut) : valeur-par-defaut est utilisée si clé n'est
            # pas présente
            "date": item["isidore"]["date"].get("normalizedDate", "0000-00-00"),
            "author": []
        })
        # Les auteurs peuvent être plusieurs : dans ce cas, on a une liste sur laquelle on bouclera
        # On utilise items[-1] car il s'agit du dernier item ajouté
        if isinstance(item["isidore"]["enrichedCreators"]["creator"], list):
            for author in item["isidore"]["enrichedCreators"]["creator"]:
                items[-1]["author"].append(author["@normalizedAuthor"])
        else:
            items[-1]["author"].append(item["isidore"]["enrichedCreators"]["creator"]["@normalizedAuthor"])

        # Des fois, le titre est un dictionnaire aussi ou une liste
        if isinstance(items[-1]["title"], dict):
            items[-1]["title"] = items[-1]["title"]["$"]
        if isinstance(items[-1]["title"], list):
            items[-1]["title"] = items[-1]["title"][0]
            # Et des fois, dans cette liste de titre, on a aussi des dictionnaires...
            if isinstance(items[-1]["title"], dict):
                items[-1]["title"] = items[-1]["title"]["$"]

    return nb_items, total_page, items, data["response"]["replies"]["page"].get("@next", None)


def cherche_isidore(q, full=False, page=1):
    """ Chercher sur isidore en faisant une requête

    :param q: Chaine de recherche
    :type q: str
    :param full: Recherche complète (itère sur toutes les pages)
    :type full: bool
    :param page: Page à récupérer
    :type page: int
    :returns: Tuple (
        Nombre de Résultats,
        Nombre de Pages,
        Liste de résultat sous forme de dictionnaire {uri, title, desc, author, date}
    )
    """

    # On exécute la requête
    params = {"output": "json", "q": q, "page": page}
    req = requests.get(ISIDORE, params=params)

    # On la parse
    nb_items, total_page, items, next_page = parser_reponse_isidore(req.json())

    if full and next_page:
        # On la parse
        nb_items, total_page, new_items, next_page = cherche_isidore(q=q, full=full, page=next_page)
        # On ajoute chacune des valeurs d'items à total_items
        items.extend(new_items)

    return nb_items, total_page, items, next_page
    
def run():
    """ Commande que l'on mettra comme commande principale
    """
    print("Commande exécutée !")
    
# Si ce fichier est le fichier exécuté directement par python
# Alors on exécute la commande
if __name__ == "__main__":
    run() 
```

Il s'agit ici de la base que nous utiliserons. Pour transformer la commande en commande pour `Click`, on fera simplement :

```python
# ...
@click.command()
def run():
    """ ....
    """
# ...
```

La syntaxe `@click.command` est ce qu'on appelle un décorateur. C'est une fonction qui transforme la fonction définie après celle-ci. C'est une manière plus rapide d'écrire quelque chose qui ressemblerait à cela (à peu près):

```python
def run():
    # Do something
    
run = click.command(run)
```

On peut utiliser autant de décorateurs que l'on veut sur une fonction ! C'est d'ailleurs ce que nous ferons.
Pour la suite, sachez que le contenu de documentation de la fonction `run()` sera utilisé comme message d'aide. Ainsi :

```
@click.command()
def run(query, full, output_file):
    """ Exécute une recherche sur Isidore.science 
    """
```

lors de l'exécution de `python recherche.py --help` affichera

```shell
Usage: recherche.py

  Exécute une recherche sur Isidore.science

Options:
  --help                 Show this message and exit.
```

## Paramètre

Notre fonction de recherche pour l'instant est particulièrement pauvre. Nous allons donc ajouter un paramètre obligatoire, appelé `argument` dans click.

```python
@click.command()
@click.argument("query", type=str)
def run(query):
    """ Exécute une recherche sur Isidore.science en utilisant [QUERY]
    """
    nb_items, total_page, items, next_page = cherche_isidore(query)
    print("Nombre de résultats : {}".format(nb_items))
    print("Nombre de résultats affichés : {}".format(len(items)))
    for item in items:
        print("{}; {}".format(item["title"], "& ".join(item["author"])))
```

et on pourra désormais exécuter `python recherche.py "Groupe Manouchian"` !

### Que remarque-t-on ?

1. L'argument se place après la déclaration de commande, en décorateur
2. Le nom de l'argument correspond au paramètre de la fonction.
3. On peut forcer un type : ici, une chaîne de caractère.


## Option

```python
@click.command()
@click.argument("query", type=str)
@click.option("-f", "--full",  is_flag=True, default=False,
              help="Browse every page of results")
def run(query, full):
    """ Exécute une recherche sur Isidore.science en utilisant [QUERY]
    """
    nb_items, total_page, items, next_page = cherche_isidore(query, full=full)
    print("Nombre de résultats : {}".format(nb_items))
    print("Nombre de résultats affichés : {}".format(len(items)))
    for item in items:
        print("{}; {}".format(item["title"], "& ".join(item["author"])))
```

et on pourra désormais exécuter `python recherche.py "Philaenis" --full` !

### Que remarque-t-on ?

1. L'option se place après la déclaration de commande, en décorateur. Elle peut arriver avant ou après un argument, mais ordonner de l'obligatoire à l'optionnel est à privilégier.
2. On peut décliner le nom de l'option en trois versions, à la suite : 
    1. En nom court (une lettre, majuscule ou minuscule, précédée **d'un** tiret.): `-q`
    2. En nom long (plusieurs lettres, majuscules ou minuscules, précédées de **deux** tirets): `--query`
    3. Avec son nom de variable, qui sera passé à la fonction. Sinon, c'est la transposition du plus long de 1) et 2)
3. On peut forcer un type : ici, un booléen
4. On utilise l'option `is_flag=True` qui permet de faire la chose suivante : la présence de `--full` donnera la valeur `True` à la variable `full`. Le défaut `default=False` assure de la valeur False en cas contraire.
5. On peut fournir une aide pour décrire la variable : regardez le résultat de `python recherche.py --help` :

```shell
Usage: isidore.py [OPTIONS] QUERY

  Exécute une recherche sur Isidore.science en utilisant [QUERY]

Options:
  -f, --full             Browse every page of results
  --help                 Show this message and exit.
```

## Typage de fichier

Il existe de [nombreux types dans `click`](http://click.palletsprojects.com/en/7.x/parameters/#parameter-types) dont le type `File` qui permet de gérer l'ouverture et la fermeture d'un fichier :

```python
@click.command()
@click.argument("query", type=str)
@click.option("-f", "--full",  is_flag=True, default=False,
              help="Browse every page of results")
@click.option("-o", "--output", "output_file", type=click.File(mode="w"), default=None,
              help="File in which to write, in a CSV manner, the results")
def run(query, full, output_file):
    """ Exécute une recherche sur Isidore.science en utilisant [QUERY]
    """
    nb_items, total_page, items, next_page = cherche_isidore(query, full=full)
    print("Nombre de résultats : {}".format(nb_items))
    print("Nombre de résultats affichés : {}".format(len(items)))
    for item in items:
        print("{}; {}".format(item["title"], "& ".join(item["author"])))

    if output_file:
        writer = csv.writer(output_file)
        writer.writerow(["date", "title", "author", "uri"])
        for item in items:
            writer.writerow([item["date"], item["title"], ", ".join(item["author"]), item["uri"]])
```

### Que remarque-t-on ?

1. Le type de fichier `click.File` prend les mêmes modes que with `open()`.
2. Le fichier est déjà ouvert par `click`.
3. Le fichier est automatiquement fermé par `click`.

## Groupe de commandes

On peut aussi ajouter plusieurs commandes dans un même fichier, généralement, on utilise pour cela un groupe :

```python

@click.group()
def group():
    """ Groupes de commandes pour communiquer avec Isidore"""


@group.command("search")
@click.argument("query", type=str)
@click.option("-f", "--full",  is_flag=True, default=False,
              help="Browse every page of results")
@click.option("-o", "--output", "output_file", type=click.File(mode="w"), default=None,
              help="File in which to write, in a CSV manner, the results")
def run(query, full, output_file):
    """ Exécute une recherche sur Isidore.science en utilisant [QUERY]
    """
    nb_items, total_page, items, next_page = cherche_isidore(query, full=full)
    print("Nombre de résultats : {}".format(nb_items))
    print("Nombre de résultats affichés : {}".format(len(items)))
    for item in items:
        print("{}; {}".format(item["title"], "& ".join(item["author"])))

    if output_file:
        writer = csv.writer(output_file)
        writer.writerow(["date", "title", "author", "uri"])
        for item in items:
            writer.writerow([item["date"], item["title"], ", ".join(item["author"]), item["uri"]])


# Si ce fichier est le fichier exécuté directement par python
# Alors on exécute la commande
if __name__ == "__main__":
    group()
```

### Que remarque-t-on ?

1. Le groupe est créé par le décorateur `click.group()` et par une fonction au nom libre, ici `mon_groupe`
2. C'est désormais cette fonction qui est utilisée à la fin
3. La commande, précédemment sans nom, se décore désormais avec `@mon_groupe.command("search")` où :
    1. le décorateur par de la fonction de groupe plutôt que de `click`.
    2. Le paramètre correspond au nom de la commande
    
On peut donc désormais faire : `python cli/isidore.py search "Philaenis" --full`

## Des librairies utiles pour ce genre de CLI

### Les couleurs

On peut facilement ajouter des couleurs ! Les librairies telles que [colour](https://pypi.org/project/colour/) sont particulièrement adaptées.

### Les tableaux

On peut facilement formater des tableaux avec des librairies comme [terminaltables](https://pypi.org/project/terminaltables/)

## Exercice

1. Ajoutez des couleurs (n'oubliez pas d'installer les librairies !)
2. Ajoutez une commande qui afficherait les mots-clefs disponibles avec une recherche (cf. https://isidore.science/vocabulary/search?q=web&output=json) (Ne vous embêtez pas avec la récursivité).

